# Bloom Filters (APS project)

### This project contains two applications of Bloom Filters namely 
*  cache based search
* username validation

## Section 1 : basic Bloom Filter

In [3]:
import math 
import mmh3 
from bitarray import bitarray

In [6]:
class BloomFilter(object):
    def __init__(self, items_count,fp_prob): 
        self.fp_prob = fp_prob 
        self.size = self.get_size(items_count,fp_prob) 
        self.hash_count = self.get_hash_count(self.size,items_count) 
        self.bit_array = bitarray(self.size) 
        self.bit_array.setall(0) 

    def add(self, item):
        digests = [] 
        for i in range(self.hash_count): 
            digest = mmh3.hash(item,i) % self.size 
            digests.append(digest)
            self.bit_array[digest] = True

    def check(self, item): 
        for i in range(self.hash_count): 
            digest = mmh3.hash(item,i) % self.size 
            if self.bit_array[digest] == False:
                return False
        return True

    @classmethod
    def get_size(self,n,p):
        m = -(n * math.log(p))/(math.log(2)**2) 
        return int(m) 

    @classmethod
    def get_hash_count(self, m, n):
        k = (m/n) * math.log(2) 
        return int(k)

In [7]:
# from bloomfilter import BloomFilter 
from random import shuffle 

n = 20 #no of items to add 
p = 0.05 #false positive probability 

bloomf = BloomFilter(n,p) 
print("Size of bit array:{}".format(bloomf.size)) 
print("False positive Probability:{}".format(bloomf.fp_prob)) 
print("Number of hash functions:{}".format(bloomf.hash_count)) 

# words to be added 
word_present = ['abound','abounds','abundance','abundant','accessable', 
                'bloom','blossom','bolster','bonny','bonus','bonuses', 
                'coherent','cohesive','colorful','comely','comfort', 
                'gems','generosity','generous','generously','genial'] 

# word not added 
word_absent = ['bluff','cheater','hate','war','humanity', 
            'racism','hurt','nuke','gloomy','facebook', 
            'geeksforgeeks','twitter'] 

for item in word_present: 
    bloomf.add(item) 

shuffle(word_present) 
shuffle(word_absent) 

test_words = word_present[:10] + word_absent 
shuffle(test_words) 
for word in test_words: 
    if bloomf.check(word): 
        if word in word_absent: 
            print("'{}' is a false positive!".format(word)) 
        else: 
            print("'{}' is probably present!".format(word)) 
    else: 
        print("'{}' is definitely not present!".format(word)) 


Size of bit array:124
False positive Probability:0.05
Number of hash functions:4
'bonny' is probably present!
'facebook' is definitely not present!
'bonuses' is probably present!
'abundant' is probably present!
'gloomy' is definitely not present!
'cheater' is definitely not present!
'humanity' is definitely not present!
'war' is definitely not present!
'gems' is probably present!
'coherent' is probably present!
'racism' is definitely not present!
'abundance' is probably present!
'twitter' is a false positive!
'abounds' is probably present!
'hate' is definitely not present!
'nuke' is definitely not present!
'blossom' is probably present!
'geeksforgeeks' is definitely not present!
'genial' is probably present!
'hurt' is definitely not present!
'bluff' is definitely not present!
'comfort' is probably present!
